<p align="center">
  <a href="https://github.com/IAHispano" target="_blank">
    <img src="https://img.shields.io/badge/Thanks%20to-IAHispano-green?style=for-the-badge&logo=github" alt="Thanks to"/>
  </a>
</p>


### **Install Applio**

In [ ]:
# @title Mount Google Drive
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
# @title Clone
import os
!git clone https://github.com/IAHispano/Applio --branch 3.5.1 --single-branch
%cd /content/Applio

In [ ]:
# @title Install

!apt update -y
!apt install -y python3.11 python3.11-distutils python3.11-dev portaudio19-dev
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.11 2
!update-alternatives --set python3 /usr/bin/python3.11
from sys import path
path.append('/usr/local/lib/python3.11/dist-packages')

print("Installing requirements...")
!curl -LsSf https://astral.sh/uv/install.sh | sh
!uv pip install -q -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu128 --index-strategy unsafe-best-match
!uv pip install -q jupyter-ui-poll

from IPython.display import clear_output
clear_output()
print("Installing requirements...")
# نصب پیشنیازهای دانلود مدل
!python core.py "prerequisites" --models "True" --exe "True" --pretraineds_hifigan "True"
clear_output()
print("Finished installing requirements!")

### **Train**

In [ ]:
# @title Preprocess Dataset
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
model_name = ""  # @param {type:"string"}
dataset_path = "/content/drive/MyDrive/"  # @param {type:"string"}

# @markdown ---
# @markdown **تنظیمات اصلی پیش‌پردازش:**
sample_rate = "40k"  # @param ["32k", "40k", "48k"] {allow-input: false}
# @markdown > طول هر قطعه صوتی (به ثانیه) را برای آموزش مشخص کنید.
chunk_len = 3 # @param {type:"slider", min:0.5, max:5.0, step:0.5}

sr = int(sample_rate.rstrip("k")) * 1000
cpu_cores = 2
cut_preprocess = "Automatic"
overlap_len = 0.3
process_effects = False
noise_reduction = False
noise_reduction_strength = 0.7
normalization_mode = "none" # اضافه شده برای سازگاری با نسخه 3.5.1

print("Starting preprocessing...")
!python core.py preprocess --model_name "{model_name}" --dataset_path "{dataset_path}" --sample_rate "{sr}" --cpu_cores "{cpu_cores}" --cut_preprocess "{cut_preprocess}" --process_effects "{process_effects}" --noise_reduction "{noise_reduction}" --noise_reduction_strength "{noise_reduction_strength}" --chunk_len "{chunk_len}" --overlap_len "{overlap_len}" --normalization_mode "{normalization_mode}"

In [ ]:
# @title Extract Features
# @markdown ### روش‌های اصلی استخراج
f0_method = "rmvpe"  # @param ["crepe", "crepe-tiny", "rmvpe"] {allow-input: false}
embedder_model = "contentvec" # @param ["contentvec", "chinese-hubert-base", "japanese-hubert-base", "korean-hubert-base", "custom"] {allow-input: false}
embedder_model_custom = "" # @param {type:"string"}

# پارامتر hop_length در نسخه‌های جدید حذف شده است
cpu_cores = 2
include_mutes = 2

sr = int(sample_rate.rstrip("k")) * 1000

print("Starting feature extraction...")
!python core.py extract --model_name "{model_name}" --f0_method "{f0_method}" --sample_rate "{sr}" --cpu_cores "{cpu_cores}" --gpu "0" --embedder_model "{embedder_model}" --embedder_model_custom "{embedder_model_custom}"  --include_mutes "{include_mutes}"

In [ ]:
#@title 🔄 بارگذاری مدل از درایو ( فقط جهت ارتقای مدلهای قبلی استفاده میشود)
import os
import shutil
from google.colab import drive

#@markdown ---
#@markdown ### نام مدلی که می‌خواهید ادامه دهید را وارد کنید:
model_name = "" #@param {type:"string"}
#@markdown ---

drive.mount('/content/drive')

if not model_name.strip():
    print("❌ لطفا نام مدل را در فیلد بالا وارد کنید.")
else:
    # مسیر بکاپ در گوگل درایو (مطابق با سلول ذخیره‌سازی شما)
    src_folder = f'/content/drive/MyDrive/RVC_backup/{model_name}'
    # مسیر مورد نیاز Applio در محیط کاری Colab
    dst_folder = f'/content/Applio/logs/{model_name}'

    print(f"در حال جستجو برای مدل '{model_name}' در گوگل درایو...")

    if os.path.exists(src_folder):
        print(f"✅ مدل پیدا شد! در حال کپی کردن فایل‌ها از:\n  > {src_folder}\nبه:\n  > {dst_folder}")

        # اگر پوشه مقصد از قبل وجود داشت، آن را پاک می‌کنیم تا فایل‌های جدید جایگزین شوند
        if os.path.exists(dst_folder):
            shutil.rmtree(dst_folder)

        # کپی کردن کل پوشه مدل
        try:
            shutil.copytree(src_folder, dst_folder)
            print(f"\n✅✅✅ مدل '{model_name}' با موفقیت در محیط کاری بارگذاری شد.")
            print("اکنون می‌توانید به سلول 'Start Training' بروید و ادامه تمرین را شروع کنید.")
        except Exception as e:
            print(f"خطا در هنگام کپی کردن فایل‌ها: {e}")

    else:
        print(f"❌ خطا: پوشه مدل در مسیر زیر پیدا نشد:\n  > {src_folder}")
        print("لطفا از درست بودن نام مدل و وجود بکاپ آن در گوگل درایو مطمئن شوید.")

In [ ]:

#@title ⚙️ Start Training (Customized & Smart v2)
import threading
import time
import os
import json

# --- بخش فرم‌ها و تنظیمات کاربر ---

# @markdown ### 💾 پشتیبان‌گیری و نمایشگر
auto_backups = False  # @param{type:"boolean"}
tensorboard = True  # @param{type:"boolean"}

# @markdown ---
# @markdown ### ⚙️ تنظیمات اصلی آموزش
total_epoch = 25  # @param {type:"integer"}
batch_size = 8  # @param {type:"slider", min:1, max:25, step:0}

# @markdown ---
# @markdown ### 📦 تنظیمات ذخیره‌سازی
save_every_epoch = 10  # @param {type:"slider", min:1, max:100, step:0}
save_only_latest = True  # @param{type:"boolean"}

# @markdown ---
# @markdown ### 🎓 قابلیت‌های پیشرفته
overtraining_detector = True  # @param{type:"boolean"}
overtraining_threshold = 50  # @param {type:"slider", min:1, max:100, step:0}
custom_pretrained = False  # @param{type:"boolean"}
g_pretrained_path = ""  # @param {type:"string"}
d_pretrained_path = ""  # @param {type:"string"}

# @markdown ### 🛠️ تنظیم دستی (sample rate)

manual_sample_rate = ""  # @param {type:"string"}


# --- تنظیمات پیش‌فرض ---
cooldown = 15
pretrained = True
cleanup = False
cache_data_in_gpu = False
save_every_weights = False
vocoder = "HiFi-GAN"
checkpointing = False
gpu = 0

# ------------------- NEW: Smart Sample Rate Detection v2 -------------------
print("در حال تحلیل حالت تمرین...")
sr = 40000  # مقدار پیش‌فرض

if manual_sample_rate.strip().isdigit():
    sr = int(manual_sample_rate)
    print(f"✅ سمپل ریت به صورت دستی روی {sr} Hz تنظیم شد.")
elif custom_pretrained and g_pretrained_path.strip():
    print("حالت «ادامه تمرین» شناسایی شد. تلاش برای خواندن سمپل ریت از فایل کانفیگ مدل...")
    config_path = os.path.join(os.path.dirname(g_pretrained_path), "config.json")

    if os.path.exists(config_path):
        try:
            with open(config_path, "r", encoding='utf-8') as f:
                config = json.load(f)
                if "sample_rate" in config:
                    sr = int(config["sample_rate"])
                    print(f"✅ سمپل ریت با موفقیت از فایل کانفیگ خوانده شد: {sr} Hz")
                else:
                    print(f"⚠️ کلید 'sample_rate' در فایل config.json پیدا نشد. لطفاً از فیلد تنظیم دستی استفاده کنید.")
        except Exception as e:
            print(f"❌ خطا در خواندن فایل config.json: {e}. لطفاً از فیلد تنظیم دستی استفاده کنید.")
    else:
        print(f"⚠️ فایل config.json در مسیر {config_path} پیدا نشد. لطفاً از فیلد تنظیم دستی استفاده کنید.")

else:
    print("حالت «تمرین جدید» شناسایی شد. تلاش برای خواندن سمپل ریت از سلول Preprocess...")
    try:
        sr = int(sample_rate.rstrip("k")) * 1000
        print(f"✅ سمپل ریت از سلول Preprocess تنظیم شد: {sr} Hz")
    except NameError:
        print(f"⚠️ متغیر sample_rate تعریف نشده است. از مقدار پیش‌فرض {sr} Hz استفاده می‌شود.")
# -------------------------------------------------------------------------


def start_train():
    if tensorboard:
        %load_ext tensorboard
        %tensorboard --logdir /content/Applio/logs/

    !python core.py train --model_name "{model_name}" --save_every_epoch "{save_every_epoch}" --save_only_latest "{save_only_latest}" --save_every_weights "{save_every_weights}" --total_epoch "{total_epoch}" --sample_rate "{sr}" --batch_size "{batch_size}" --gpu "{gpu}" --pretrained "{pretrained}" --custom_pretrained "{custom_pretrained}" --g_pretrained_path "{g_pretrained_path}" --d_pretrained_path "{d_pretrained_path}" --overtraining_detector "{overtraining_detector}" --overtraining_threshold "{overtraining_threshold}" --cleanup "{cleanup}" --cache_data_in_gpu "{cache_data_in_gpu}" --vocoder "{vocoder}" --checkpointing "{checkpointing}"

    print("\n" + "="*50)
    print("✅ آموزش به پایان رسید. اکنون می‌توانید سلول را متوقف کنید.")
    print("="*50 + "\n")

    if overtraining_detector:
        print("📜 گزارش نهایی (با شناساگر بیش‌برازش فعال):")
        print("- آموزش یا به تعداد ایپاک کامل (total_epoch) رسیده یا به دلیل شناسایی بیش‌برازش زودتر متوقف شده است.")
    else:
        print("📜 گزارش نهایی (با شناساگر بیش‌برازش غیرفعال):")
        print("- آموزش تا آخرین ایپاک مشخص‌شده (total_epoch) ادامه یافت.")
    print("\n" + "="*50)


server_thread = threading.Thread(target=start_train)
server_thread.start()

if auto_backups:
    print("AutoBackup Enabled. Starting backup loop...")
    while server_thread.is_alive():
        time.sleep(cooldown)
else:
    print("AutoBackup Disabled. The training will run without backups.")
    while server_thread.is_alive():
        time.sleep(10)

print("فرآیند اصلی سلول به پایان رسید.")

In [ ]:
#@title Save
model_name = "" #@param {type:"string"}

import os
import shutil
from google.colab import drive

drive.mount('/content/drive')

src_folder = f'/content/Applio/logs/{model_name}'
dst_folder = f'/content/drive/MyDrive/RVC_backup/{model_name}'

os.makedirs(dst_folder, exist_ok=True)

for root, dirs, files in os.walk(src_folder):
    relative_path = os.path.relpath(root, src_folder)
    dst_path = os.path.join(dst_folder, relative_path)
    os.makedirs(dst_path, exist_ok=True)

    for file in files:
        src_file = os.path.join(root, file)
        dst_file = os.path.join(dst_path, file)
        try:
            shutil.copy2(src_file, dst_file)
            print(f'کپی شد: {src_file} → {dst_file}')
        except Exception as e:
            print(f'خطا در کپی "{src_file}": {e}')

print("✅ همه فایل‌ها با موفقیت منتقل شدند.")

In [ ]:

#@title ذخیره بصورت زیپ
import os
import re
import zipfile
from google.colab import drive

# --- این بخش را ویرایش کنید ---
#@markdown ### نام مدلی که می‌خواهید ذخیره کنید   :
model_name = "" #@param {type:"string"}
# -----------------------------

drive.mount('/content/drive')

if not model_name.strip():
    print("❌ لطفا نام مدل را وارد کنید.")
else:
    # تعریف مسیرها
    model_dir = f'/content/Applio/logs/{model_name}'
    drive_root = '/content/drive/MyDrive'
    zip_path = f'{drive_root}/{model_name}_final_model.zip'

    print(f"جستجو برای فایل‌ها در پوشه: {model_dir}")

    # پیدا کردن جدیدترین فایل مدل 'best_epoch.pth'
    latest_epoch = -1
    latest_model_file = None

    if not os.path.isdir(model_dir):
        print(f"❌ خطا: پوشه مدل '{model_dir}' پیدا نشد.")
    else:
        for filename in os.listdir(model_dir):
            # فقط فایل‌هایی که با _best_epoch.pth تمام می‌شوند را بررسی کن
            if filename.endswith("_best_epoch.pth"):
                match = re.search(r'_(\d+)e_', filename)
                if match:
                    current_epoch = int(match.group(1))
                    if current_epoch > latest_epoch:
                        latest_epoch = current_epoch
                        latest_model_file = filename

    # پیدا کردن فایل ایندکس
    index_file_name = f'{model_name}.index'
    index_file_path = os.path.join(model_dir, index_file_name)

    # بررسی نهایی و ایجاد فایل زیپ
    if latest_model_file and os.path.exists(index_file_path):
        latest_model_path = os.path.join(model_dir, latest_model_file)

        print(f"✅ فایل مدل نهایی: {latest_model_file}")
        print(f"✅ فایل ایندکس: {index_file_name}")
        print(f"در حال ایجاد فایل زیپ در مسیر: {zip_path}")

        try:
            with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
                zipf.write(latest_model_path, arcname=latest_model_file)
                zipf.write(index_file_path, arcname=index_file_name)

            print(f"\n✅✅✅ فایل '{os.path.basename(zip_path)}' با موفقیت در صفحه اصلی گوگل درایو شما ذخیره شد.")

        except Exception as e:
            print(f"❌ خطا در هنگام ایجاد فایل زیپ: {e}")

    else:
        if not latest_model_file:
            print("❌ خطا: هیچ فایل مدلی با پسوند '_best_epoch.pth' در پوشه مدل پیدا نشد.")
        if not os.path.exists(index_file_path):
            print(f"❌ خطا: فایل ایندکس '{index_file_name}' در پوشه مدل پیدا نشد.")

<p align="center">
  <a href="https://colab.research.google.com/github/yaranbarzi/Applio/blob/main/Inference_2025.ipynb" target="_blank">
    <img src="https://img.shields.io/badge/Inference%20RVC-Open%20in%20Colab-F9AB00?style=for-the-badge&logo=googlecolab&logoColor=white" alt="Inference RVC"/>
  </a>
</p>


<p align="center">
  <a href="https://youtube.com/@aigolden" target="_blank">
    <img src="https://img.shields.io/badge/YouTube-aigolden-FF0000?style=for-the-badge&logo=youtube&logoColor=white" alt="YouTube Channel"/>
  </a>
  &nbsp;&nbsp;
  <a href="https://t.me/ai_golden" target="_blank">
    <img src="https://img.shields.io/badge/Telegram-ai__golden-26A5E4?style=for-the-badge&logo=telegram&logoColor=white" alt="Telegram Channel"/>
  </a>
</p>